In [29]:
import requests
import re
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import sys
import matplotlib.pyplot as plt
from collections import Counter
import xml.etree.ElementTree as ET
from typing import List, Dict, Any, Optional
import duckdb
import unicodedata
import importlib
import numpy as np

from pathlib import Path
import re
import pandas as pd

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import grela_helpers

In [3]:
conn = grela_helpers.make_connection()

In [4]:
#grela_extraction_helpers.conn.close()
# importlib.reload(grela_extraction_helpers)

In [5]:
# establish connection with Google sheets
file_data = json.load(open(os.path.expanduser("../../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ruland_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1od248fJpNbwMO8IVZQVO3xnzrmmTe68yS9XWajyuk5o/edit?usp=sharing")

In [6]:
lexeme_df = pd.read_parquet("../data/ruland-dictionaries.parquet")

In [7]:
lexeme_df["target_len"] = lexeme_df["target_canonical"].str.split().str.len()

In [8]:
lexeme_df = lexeme_df[lexeme_df["target_relemmatized"].str.split().str.len() < 4]

In [9]:
MAX_N = 3
lexeme_df = lexeme_df[lexeme_df["target_len"] <= MAX_N].copy()
len(lexeme_df)

2846

In [10]:
# look at a few multiword entries
lexeme_df[lexeme_df["target_len"] > 1].head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
23,Lapis maior,[Lapis maior],lapis maior,"[Ruland1612-Lapis-maior, Ruland1612-Lapis-maio...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Lapis-m...",3,"[finde nach etlicher Meynung, die calcinirten ...","[None, None, id est, der Geist außgezogen auß ...","[finde nach etlicher Meynung, die calcinirten ...","[{'entry_id': 'Ruland1612-Lapis-maior', 'quote...",2,lapis magnus
24,Leo viridis,[Leo viridis],leo uiridis,"[Ruland1612-Leo-viridis, Ruland1612-Leo-viridi...",3,[L],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Leo-vir...",3,"[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[None, quorundam opinione aurum, vitriolum]","[ist Hermetis Erz Glaß ond vitriol, vnd das Bl...","[{'entry_id': 'Ruland1612-Leo-viridis', 'quote...",2,leo uiridis
25,Machina tractoria,[Machina tractoria],machina tractoria,"[Ruland1612-Machina-tractoria, Ruland1612-Mach...",3,[M],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Machina...",3,"[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[None, None, None]","[Haspel, Gezeugsso Derg ond Waͤsser heben., Sc...","[{'entry_id': 'Ruland1612-Machina-tractoria', ...",2,machina tractoria
30,Panum aereorum species,[Panum aereorum species],panum aereorum species,"[Ruland1612-Panum-aereorum-species-,-Vorpleits...",3,[P],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Panum-a...",3,"[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...","[None, None, None]","[Vorpleitstein, Vorpleitloch., Lech, zwir Vorp...",[{'entry_id': 'Ruland1612-Panum-aereorum-speci...,3,panis aereus species
32,Pyritae aurei coloris,[Pyritae aurei coloris],pyritae aurei coloris,[Ruland1612-Pyritae-aurei-coloris-.-1.-Pyrites...,3,[P],[],[],"[{'def': '1', 'entry_id': 'Ruland1612-Pyritae-...",3,"[1, è quibus ignis elicitur, è quibus ignis no...","[1, è quibus ignis elicitur, è quibus ignis no...",[1. Pyrites qui est solidus & colorem auri tot...,[{'entry_id': 'Ruland1612-Pyritae-aurei-colori...,3,pyritus aureus color


In [11]:
# look at entries starting with mercurius
lexeme_df[lexeme_df["target_canonical"].str.startswith("mercurius")]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized
27,Mercurius,[Mercurius],mercurius,"[Ruland1612-Mercurius, Ruland1612-Mercurius, R...",3,[M],[],[],"[{'def': 'id est, sulphur.', 'entry_id': 'Rula...",3,"[id est, sulphur., est principium materiale, v...","[id est, sulphur., est principium materiale, v...","[id est, sulphur. Mercurius. Mercurius ist in ...","[{'entry_id': 'Ruland1612-Mercurius', 'quote':...",1,mercurius
96,Mercurius metallorum,[Mercurius metallorum],mercurius metallorum,"[Ruland1612-Mercurius-metallorum, Ruland1612-M...",2,[M],[],[],[{'def': 'Ist darauß die Natur der Coͤrper gez...,2,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[Ist darauß die Natur der Coͤrper gezogen wird...,[{'entry_id': 'Ruland1612-Mercurius-metallorum...,2,mercurius metallum
1880,Mercurius argentipigmentum,[Mercurius argentipigmentum],mercurius argentipigmentum,[Ruland1612-Mercurius-argentipigmentum],1,[M],[],[],"[{'def': 'ist Schwefel victriol, Alaun Saltz-d...",1,"[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...","[ist Schwefel victriol, Alaun Saltz-dieweils d...",[{'entry_id': 'Ruland1612-Mercurius-argentipig...,2,mercurius argentipigmentum
1881,Mercurius chambar,[Mercurius chambar],mercurius chambar,[Ruland1612-Mercurius-chambar],1,[M],[],[],"[{'def': 'ist magnesia, fuessend Coͤrper und W...",1,"[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[ist magnesia, fuessend Coͤrper und Wasser.]","[{'entry_id': 'Ruland1612-Mercurius-chambar', ...",2,mercurius chaambar
1882,Mercurius corallinus,[Mercurius corallinus],mercurius corallinus,[Ruland1612-Mercurius-corallinus],1,[M],[],[],[{'def': 'qui per oleum ouorum & aquas alias i...,1,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[qui per oleum ouorum & aquas alias in rubedin...,[],2,mercurius corallinus
1883,Mercurius crudus,[Mercurius crudus],mercurius crudus,[Ruland1612-Mercurius-crudus],1,[M],[],[],"[{'def': 'est is, qui nondum separatus est a s...",1,"[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[est is, qui nondum separatus est a sua matric...","[{'entry_id': 'Ruland1612-Mercurius-crudus', '...",2,mercurius crudus
1884,Mercurius crystallinus,[Mercurius crystallinus],mercurius crystallinus,"[Ruland1612-Mercurius-crystallinus,]",1,[M],[],[],[{'def': 'qui saepe sublimatus est in formam c...,1,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[qui saepe sublimatus est in formam crystalli ...,[],2,mercurius crystallinus
1885,Mercurius laxus,[Mercurius laxus],mercurius laxus,[Ruland1612-Mercurius-laxus],1,[M],[],[],"[{'def': 'est turbith minerale.', 'entry_id': ...",1,[est turbith minerale.],[est turbith minerale.],[est turbith minerale.],[],2,mercurius laxus
1886,Mercurius mineralium,[Mercurius mineralium],mercurius mineralium,[Ruland1612-Mercurius-mineralium],1,[M],[],[],"[{'def': 'est oleitas, vel vnctuositas de mine...",1,"[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...","[est oleitas, vel vnctuositas de mineris auri ...",[{'entry_id': 'Ruland1612-Mercurius-mineralium...,2,mercurius mineralis
1887,Mercurius regeneratus,[Mercurius regeneratus],mercurius regeneratus,[Ruland1612-Mercurius-regeneratus],1,[M],[],[],"[{'def': 'est primum ens Mercurii.', 'entry_id...",1,[est primum ens Mercurii.],[est primum ens Mercurii.],[est primum ens Mercurii.],[],2,mercurius regenero


In [12]:
# test with one specific multiword phrase
target_canonical     =  "lapis philosophorum" # "lapis philosophorum"
target_relemmatized  =  "lapis philosophus" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()

(603,)

In [14]:
# 2) phase 2: build detailed KWIC for first N hits
df = grela_helpers.kwic_from_spans(
    conn,
    spans_table="temp_target_spans_table",
    window=10,
    include_tokens=True,
    max_hits=100_000,     # or None, if you know it's safe
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352412024, 352412025]","[10, 11]","[17, 18]",emlap_100002,emlap_100002_162,emlap_100002_162,emlap_100002_162,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 352412014, 'token_text': ',', 'l...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 352412007, 'token_text': 'Et', '...",Pseudo-Lull,De secretis naturae,1518,1518
1,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352425603, 352425604]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_750,emlap_100002_749,emlap_100002_750,quia ibi large continetur hec doctrina . ¶item...,"[{'token_id': 352425593, 'token_text': 'quia',...",¶item in compositione lapidis philosophorum ru...,"[{'token_id': 352425600, 'token_text': '¶item'...",Pseudo-Lull,De secretis naturae,1518,1518
2,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426859, 352426860]","[10, 11]","[3, 4]",emlap_100002,emlap_100002_800,emlap_100002_798,emlap_100002_801,"& uirtutem sue nature . ¶questio . Pater , si ...","[{'token_id': 352426849, 'token_text': '&', 'l...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 352426856, 'token_text': 'Pater'...",Pseudo-Lull,De secretis naturae,1518,1518
3,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426881, 352426882]","[10, 11]","[14, 15]",emlap_100002,emlap_100002_801,emlap_100002_801,emlap_100002_801,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 352426871, 'token_text': 'non', ...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 352426867, 'token_text': 'Fili',...",Pseudo-Lull,De secretis naturae,1518,1518
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353269572, 353269573]","[10, 11]","[3, 4]",emlap_100004,emlap_100004_10754,emlap_100004_10752,emlap_100004_10756,ordinem . Incipiunt capitula primi libri . Quo...,"[{'token_id': 353269562, 'token_text': 'ordine...",Quod unus est lapis philosophorum Cap .,"[{'token_id': 353269569, 'token_text': 'Quod',...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1, 2]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1, 2]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5, 6]",noscemus_906984,noscemus_906984_17646,noscemus_906984_17645,noscemus_906984_17649,"655 . pl. 31/2 . Hemb , Matthias , de Lapide p...","[{'token_id': 338690964, 'token_text': '655', ...","Hemb , Matthias , de Lapide philosophorum prim...","[{'token_id': 338690969, 'token_text': 'Hemb',...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
601,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, ph

In [22]:
# test with one specific multiword phrase
target_canonical     =  "aqua" # "lapis philosophorum"
target_relemmatized  =  "aqua" # "lapis philosophus"
# 1) phase 1: spans only (whole GreLa, but capped)
grela_helpers.create_target_spans_table(
    conn,
    table_name="temp_target_spans_table",
    target_canonical=target_canonical,
    target_relemmatized=target_relemmatized,
    emlap_only=False,
    max_hits=1_000_000,   # safeguard
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
n_spans = conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()[0]
n_spans

223891

In [26]:
batch_size = 100_000
if n_spans > batch_size:
    all_chunks = []

    for start_idx in range(1, n_spans + 1, batch_size):
        end_idx = min(start_idx + batch_size - 1, n_spans)
        print(f"Processing spans {start_idx}–{end_idx}...")

        df_chunk = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
            span_idx_min=start_idx,
            span_idx_max=end_idx,
        )

        all_chunks.append(df_chunk)

    df = pd.concat(all_chunks, ignore_index=True)
else:
    df = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
        )

Processing spans 1–100000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing spans 100001–200000...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing spans 200001–223891...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
len(df)

In [47]:
def safe_name(s: str | None) -> str:
    """Make a safe short filename component from a target phrase."""
    if not s or not isinstance(s, str):
        return "unknown"
    s = s.strip().lower().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]+", "", s)   # keep only safe chars
    return s or "unnamed"

In [46]:
outdir = Path("../data/large_files/emlap_ruland_instances/")
outdir.mkdir(exist_ok=True)

In [48]:
def from_entries_to_file(tc, tr, outdir):
    name = safe_name(tc)
    fname = f"{name}.parquet"
    fpath = str(outdir / fname)
    grela_helpers.create_target_spans_table(
        conn,
        table_name="temp_target_spans_table",
        target_canonical=tc,
        target_relemmatized=tr,
        emlap_only=False,
        max_hits=1_000_000,   # safeguard
    )
    batch_size = 100_000
    n_spans = conn.execute("SELECT COUNT(*) FROM temp_target_spans_table").fetchone()[0]
    if n_spans <= batch_size:
        df = grela_helpers.kwic_from_spans(
            conn,
            spans_table="temp_target_spans_table",
            window=10,
            include_tokens=True,
            out_path=fpath,
            )

In [19]:
tc = "mercurius metallorum"
tr = "mercurius metallum"
from_entries_to_file(tc, tr, outdir)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [49]:
def read_hits(fname):
    try:
        path = os.path.join(outdir, fname)
        instances = pd.read_parquet(path).to_dict("records")
    except:
        instances = []
    return instances

In [50]:
df = read_hits("mercurius.parquet")

In [51]:
pd.DataFrame(df)

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,[Mercurius],canonical,lemma,[mercurius],[221779315],[10],[8],cc_10005,cc_10005_1205,cc_10005_1204,cc_10005_1206,est an CCXLVII Bed chr Romae Joannes secundus ...,"[{'token_id': 221779305, 'token_text': 'est', ...",CCXLVII Bed chr Romae Joannes secundus qui et ...,"[{'token_id': 221779307, 'token_text': 'CCXLVI...",Hermannus Contractus,Chronicon,1013.0,1054.0
1,[Mercurius],canonical,lemma,[mercurius],[221819279],[10],[8],cc_10005,cc_10005_4223,cc_10005_4222,cc_10005_4224,est an CCXLVII Bed chr Romae Joannes secundus ...,"[{'token_id': 221819269, 'token_text': 'est', ...",CCXLVII Bed chr Romae Joannes secundus qui et ...,"[{'token_id': 221819271, 'token_text': 'CCXLVI...",Hermannus Contractus,Chronicon,1013.0,1054.0
2,[Mercurii],canonical,lemma,[mercurius],[51757983],[10],[34],cc_10031,cc_10031_43,cc_10031_43,cc_10031_43,ita constitutam et dispositam videlicet ut ab ...,"[{'token_id': 51757973, 'token_text': 'ita', '...",Recipite ergo et tenete pacem et illam trevam ...,"[{'token_id': 51757949, 'token_text': 'Recipit...",Raimbaldus Arelatensis,Diplomata,1031.0,1067.0
3,[Mercurii],canonical,lemma,[mercurius],[149133126],[10],[7],cc_10066,cc_10066_5562,cc_10066_5561,cc_10066_5563,De malis praedicatoribus 7 Sicut qui mittit la...,"[{'token_id': 149133116, 'token_text': 'De', '...",7 Sicut qui mittit lapidem in acervum Mercurii...,"[{'token_id': 149133119, 'token_text': '7', 'l...",Petrus Damianus,Collectanea in Vetus Testamentum,1007.0,1072.0
4,[Mercurii],canonical,lemma,[mercurius],[149134895],[10],[7],cc_10066,cc_10066_5654,cc_10066_5653,cc_10066_5655,In eadem epistola « Sicut qui mittit lapidem i...,"[{'token_id': 149134885, 'token_text': 'In', '...",« Sicut qui mittit lapidem in acervum Mercurii...,"[{'token_id': 149134888, 'token_text': '«', 'l...",Petrus Damianus,Collectanea in Vetus Testamentum,1007.0,1072.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21691,[Mercurius],canonical,lemma,[mercurius],[329702154],[10],[0],noscemus_971293,noscemus_971293_583,noscemus_971293_582,noscemus_971293_584,"solum , quod de reliquis non participet , repe...","[{'token_id': 329702144, 'token_text': 'solum'...",Mercurius enim continet substantiam sulphuream...,"[{'token_id': 329702154, 'token_text': 'Mercur...","Béguin, Jean",Tyrocinium chymicum recognitum et auctum. Auct...,1612.0,1612.0
21692,[Mercurii],canonical,lemma,[mercurius],[329702481],[10],[18],noscemus_971293,noscemus_971293_615,noscemus_971293_615,noscemus_971293_615,"bracteas uel laminas ductum , cum octo partibu...","[{'token_id': 329702471, 'token_text': 'bracte...","quando metallum , excepto ferro , in tenues br...","[{'token_id': 329702463, 'token_text': 'quando...","Béguin, Jean",Tyrocinium chymicum recognitum et auctum. Auct...,1612.0,1612.0
21693,[mercurii],canonical,lemma,[mercurius],[329702728],[10],[26],noscemus_971293,noscemus_971293_637,noscemus_971293_637,noscemus_971293_638,"fortes , acetum , uel uaporem plumbi liquefact...","[{'token_id': 329702718, 'token_text': 'fortes...","quod corpus metallicum in lamellas reductum , ...","[{'token_id': 329702702, 'token_text': 'quod',...","Béguin, Jean",Tyrocinium chymicum recognitum et auctum. Auct...,1612.0,1612.0
21694,[Mercurium],canonical,lemma,[mercurius],[294412214],[10],[6],vulgate_tlg0031.tlg005.obi-lat,vulgate_tlg0031.tlg005.obi-lat_500,vulgate_tlg0031.tlg005.obi-lat_499,vulgate_tlg0031.tlg005.obi-lat_501,hominibus descenderunt ad nos et vocabant Barn...,"[{'token_id': 294412204, 'token_text': 'homini...",et vocabant Barnaban Iovem Paulum vero Mercuri...,"[{'token_id': 294412208, 'token_text': 'et', '...",Luke (the evangelist),Acts,383.0,405.0


In [52]:
lexeme_df["instance_fname"] = lexeme_df["target_canonical"].apply(lambda x: safe_name(x) + ".parquet")

In [53]:
lexeme_df[lexeme_df.duplicated(subset="instance_fname", keep=False)]

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname


In [25]:
done = []

In [ ]:
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    target = tc or tr
    name = safe_name(tc)
    fname = f"{name}.parquet"
    if fname not in done:
        from_entries_to_file(tc, tr, outdir)
        done.append(fname)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
len(os.listdir(outdir))

2217

In [19]:
emlap_metadata = pd.read_csv(
        "https://raw.githubusercontent.com/CCS-ZCU/EMLAP_ETL/refs/heads/master/data/emlap_metadata.csv",
        sep=";",
    )
emlap_metadata.head(5)

,Unnamed: 0,working_title,filenames,no.,is_done,is_noscemus,if_noscemus_id,AUTHORSHIP,is_one_author,#if more than 1 author skip section and choose compendium below,...,CONTENTS,genre,subject,SOURCE OF FILE,link,source_of_file,origin_of_copy,other_notes,tokens_N,aurhor_wd
0,0,"Augurello, Chrysopoeia",100001_Augurello1515_Chrysopoeia_GB_Noscemus,100001,True,True,713324.0,NaN,True,NaN,...,NaN,didactic poem,alchemy,NaN,https://wiki.uibk.ac.at/noscemus/Chrysopoeia,GB,Noscemus,NaN,23718,NaN
1,1,"Pseudo-Lull, Secretis",100002_Pseudo-Lull1518_De secretis_naturae_MDZ...,100002,True,False,NaN,NaN,True,NaN,...,NaN,treatise,"alchemy, medicine",NaN,https://www.digitale-sammlungen.de/en/view/bsb...,MDZ,MBS,NaN,24673,NaN
2,2,"Pantheus, Ars Transmutatione",100003_Pantheus1518_Ars_Transmutationis_Metall...,100003,True,False,NaN,NaN,True,NaN,...,NaN,treatise,alchemy,NaN,https://www.google.co.uk/books/edition/Ars_Tra...,GB,BL,NaN,8646,NaN
3,3,"Anon, Vera alchemiae",100004_Anon1561_Verae_Alchemiae_MDZ_MBS,100004,True,False,NaN,NaN,True,NaN,...,NaN,"compendium, florilegium",alchemy,NaN,https://mdz-nbn-resolving.de/details:bsb10141168,MDZ,MBS,NaN,3521,NaN
4,4,"Pantheus, Voarchadumia",100005_Pantheus1530_Voarchadumia_ONB,100005,True,False,NaN,NaN,True,NaN,...,NaN,treatise,alchemy,NaN,https://data.onb.ac.at/rep/10588E49,ONB,ONB,NaN,20386,NaN


In [75]:
df = pd.DataFrame(read_hits("lapis_philosophorum.parquet"))
df

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
0,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352412024, 352412025]","[10, 11]","[17.0, 18.0]",emlap_100002,emlap_100002_162,emlap_100002_162,emlap_100002_162,", aquarum tam uegetabilium quam mineralium ad ...","[{'token_id': 352412014, 'token_text': ',', 'l...","Et cum hoc sufficit tibi modus generalis , aqu...","[{'token_id': 352412007, 'token_text': 'Et', '...",Pseudo-Lull,De secretis naturae,1518,1518
1,"[lapidis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352425603, 352425604]","[10, 11]","[3.0, 4.0]",emlap_100002,emlap_100002_750,emlap_100002_749,emlap_100002_750,quia ibi large continetur hec doctrina . ¶item...,"[{'token_id': 352425593, 'token_text': 'quia',...",¶item in compositione lapidis philosophorum ru...,"[{'token_id': 352425600, 'token_text': '¶item'...",Pseudo-Lull,De secretis naturae,1518,1518
2,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426859, 352426860]","[10, 11]","[3.0, 4.0]",emlap_100002,emlap_100002_800,emlap_100002_798,emlap_100002_801,"& uirtutem sue nature . ¶questio . Pater , si ...","[{'token_id': 352426849, 'token_text': '&', 'l...","Pater , si lapis philosophorum cadit in doctri...","[{'token_id': 352426856, 'token_text': 'Pater'...",Pseudo-Lull,De secretis naturae,1518,1518
3,"[lapidem, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[352426881, 352426882]","[10, 11]","[14.0, 15.0]",emlap_100002,emlap_100002_801,emlap_100002_801,emlap_100002_801,"non solum ad preciosos lapides , sed et iam ad...","[{'token_id': 352426871, 'token_text': 'non', ...","Fili , hec regula non solum ad preciosos lapid...","[{'token_id': 352426867, 'token_text': 'Fili',...",Pseudo-Lull,De secretis naturae,1518,1518
4,"[lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[353269572, 353269573]","[10, 11]","[3.0, 4.0]",emlap_100004,emlap_100004_10754,emlap_100004_10752,emlap_100004_10756,ordinem . Incipiunt capitula primi libri . Quo...,"[{'token_id': 353269562, 'token_text': 'ordine...",Quod unus est lapis philosophorum Cap .,"[{'token_id': 353269569, 'token_text': 'Quod',...","Gratarolo, Guilelmo",Verae Alchemiae,1561,1561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscemus_906984_17645,noscemus_906984_17649,"655 . pl. 31/2 . Hemb , Matthias , de Lapide p...","[{'token_id': 338690964, 'token_text': '655', ...","Hemb , Matthias , de Lapide philosophorum prim...","[{'token_id': 338690969, 'token_text': 'Hemb',...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
601,"[lapidis, philosophorum]

In [76]:
# check instances from noscemus
df[df["grela_id"].str.startswith("noscemus")]

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
467,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308529698, 308529699]","[10, 11]","[1.0, 2.0]",noscemus_1085290,noscemus_1085290_1138,noscemus_1085290_1135,noscemus_1085290_1139,& amaro edidit . U. Laur . Gryll . de Lapide P...,"[{'token_id': 308529688, 'token_text': '&', 'l...",de Lapide Philosophorum .,"[{'token_id': 308529697, 'token_text': 'de', '...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
468,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308632543, 308632544]","[10, 11]","[18.0, 19.0]",noscemus_1085290,noscemus_1085290_15736,noscemus_1085290_15736,noscemus_1085290_15736,"uentilata & nec dum relo luta Controuersia , a...","[{'token_id': 308632533, 'token_text': 'uentil...",Ethicochymico - medicum de illa ueteri toties ...,"[{'token_id': 308632525, 'token_text': 'Ethico...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
469,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308637520, 308637521]","[10, 11]","[4.0, 5.0]",noscemus_1085290,noscemus_1085290_16346,noscemus_1085290_16345,noscemus_1085290_16347,", 1552 , in fol . Efferrarius , Monachus De La...","[{'token_id': 308637510, 'token_text': ',', 'l...","Efferrarius , Monachus De Lapide Philosophorum...","[{'token_id': 308637516, 'token_text': 'Efferr...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
470,"[Lapide, Philosopha]",relemmatized,lemma,"[lapis, philosophus]","[308642667, 308642668]","[10, 11]","[26.0, 27.0]",noscemus_1085290,noscemus_1085290_17090,noscemus_1085290_17090,noscemus_1085290_17091,que descriptae Tius praeter illas etiam in phu...,"[{'token_id': 308642657, 'token_text': 'que', ...","Qua non paucae ab Antiquioribus Theophrasto , ...","[{'token_id': 308642641, 'token_text': 'Qua', ...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
471,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308534362, 308534363]","[10, 11]","[0.0, 1.0]",noscemus_1085290,noscemus_1085290_1823,noscemus_1085290_1820,noscemus_1085290_1824,". Uenetiis , ex Offic . Farrea , 1543 . Lapis ...","[{'token_id': 308534352, 'token_text': '.', 'l...","Lapis Philosophorum , aperiens , ac in 8 .","[{'token_id': 308534362, 'token_text': 'Lapis'...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscem

In [77]:
# find NOSCEMUY entries included in EMLAP
noscemus_emlap_ids = [str(int(x)) for x in emlap_metadata["if_noscemus_id"].unique() if pd.notna(x)]
noscemus_emlap_ids

['713324', '26765', '597737', '971293', '688159']

In [80]:
df[df["grela_id"].apply(lambda x: (x[:8] == "noscemus") and  (x[9:] not in noscemus_emlap_ids))]

,target_phrase,target_from,matched_by,target_lemmata,target_token_ids,target_kwic_idx,target_sent_idx,grela_id,target_sentence_id,start_sentence_id,end_sentence_id,kwic_text,kwic_tokens,target_sentence_text,target_sentence_tokens,author,title,not_before,not_after
467,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308529698, 308529699]","[10, 11]","[1.0, 2.0]",noscemus_1085290,noscemus_1085290_1138,noscemus_1085290_1135,noscemus_1085290_1139,& amaro edidit . U. Laur . Gryll . de Lapide P...,"[{'token_id': 308529688, 'token_text': '&', 'l...",de Lapide Philosophorum .,"[{'token_id': 308529697, 'token_text': 'de', '...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
468,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308632543, 308632544]","[10, 11]","[18.0, 19.0]",noscemus_1085290,noscemus_1085290_15736,noscemus_1085290_15736,noscemus_1085290_15736,"uentilata & nec dum relo luta Controuersia , a...","[{'token_id': 308632533, 'token_text': 'uentil...",Ethicochymico - medicum de illa ueteri toties ...,"[{'token_id': 308632525, 'token_text': 'Ethico...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
469,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308637520, 308637521]","[10, 11]","[4.0, 5.0]",noscemus_1085290,noscemus_1085290_16346,noscemus_1085290_16345,noscemus_1085290_16347,", 1552 , in fol . Efferrarius , Monachus De La...","[{'token_id': 308637510, 'token_text': ',', 'l...","Efferrarius , Monachus De Lapide Philosophorum...","[{'token_id': 308637516, 'token_text': 'Efferr...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
470,"[Lapide, Philosopha]",relemmatized,lemma,"[lapis, philosophus]","[308642667, 308642668]","[10, 11]","[26.0, 27.0]",noscemus_1085290,noscemus_1085290_17090,noscemus_1085290_17090,noscemus_1085290_17091,que descriptae Tius praeter illas etiam in phu...,"[{'token_id': 308642657, 'token_text': 'que', ...","Qua non paucae ab Antiquioribus Theophrasto , ...","[{'token_id': 308642641, 'token_text': 'Qua', ...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
471,"[Lapis, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[308534362, 308534363]","[10, 11]","[0.0, 1.0]",noscemus_1085290,noscemus_1085290_1823,noscemus_1085290_1820,noscemus_1085290_1824,". Uenetiis , ex Offic . Farrea , 1543 . Lapis ...","[{'token_id': 308534352, 'token_text': '.', 'l...","Lapis Philosophorum , aperiens , ac in 8 .","[{'token_id': 308534362, 'token_text': 'Lapis'...","Linden, Johannes Antonides van der","De scriptis medicis libri duo, quibus praemitt...",1637,1637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,"[Lapis, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[341163674, 341163675]","[10, 11]","[1.0, 2.0]",noscemus_906982,noscemus_906982_55782,noscemus_906982_55780,noscemus_906982_55784,. 3279 4701 . 8622 corruptio in nutricibus . 7...,"[{'token_id': 341163664, 'token_text': '.', 'l...","745 Lapis philosophorum , uid .","[{'token_id': 341163673, 'token_text': '745', ...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
599,"[Lapide, Philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338672111, 338672112]","[10, 11]","[1.0, 2.0]",noscemus_906984,noscemus_906984_13542,noscemus_906984_13539,noscemus_906984_13544,ib . 739 . pl. 2 . prop . de Lapide Philosopho...,"[{'token_id': 338672101, 'token_text': 'ib', '...",de Lapide Philosophorum ceu medicina uniuersal...,"[{'token_id': 338672110, 'token_text': 'de', '...","Heffter, Johann Carl","Domini Joannis Caroli Heffteri, medicinae prac...",1756,1764
600,"[Lapide, philosophorum]",relemmatized,lemma,"[lapis, philosophus]","[338690974, 338690975]","[10, 11]","[5.0, 6.0]",noscemus_906984,noscemus_906984_17646,noscem

In [98]:
csv_path = "../data/emlap_ruland_instances_csv/"
os.makedirs(csv_path, exist_ok=True)

def read_and_filter_hits(fname):
    try:
        # read instances as dataframe:
        df = pd.DataFrame(read_hits(fname))
        # count noscemus instances:
        noscemus_instances_N = df["grela_id"].apply(lambda x: (x[:8] == "noscemus") and  (x[9:] not in noscemus_emlap_ids)).sum()
        # count cc instances:
        cc_instances_N = df["grela_id"].str.startswith("cc").sum()
        # extract emlap instances as df:
        emlap_instances_df = df[df["grela_id"].str.startswith("emlap")]
        emlap_instances_df.to_csv(os.path.join(csv_path, fname.replace(".parquet", ".csv")), sep=";", index=False)
        #
        emlap_instances = emlap_instances_df.to_dict("records")
        emlap_instances_N = len(emlap_instances)
        return {"noscemus_instances_N": noscemus_instances_N, "cc_instances_N": cc_instances_N, "emlap_instances_N": emlap_instances_N, "emlap_instances": emlap_instances}
    except:
        return {"noscemus_instances_N": 0, "cc_instances_N": 0, "emlap_instances_N": 0, "emlap_instances": []}


In [99]:
instances_dict_series = pd.DataFrame(lexeme_df["instance_fname"].sample(10).apply(read_and_filter_hits).tolist())

In [100]:
instances_dict_series

,noscemus_instances_N,cc_instances_N,emlap_instances_N,emlap_instances
0,5,0,0,[]
1,1,0,3,"[{'target_phrase': ['separatione', 'per', 'abs..."
2,77,77,2,"[{'target_phrase': ['Etesiae'], 'target_from':..."
3,85,0,3,"[{'target_phrase': ['Karabe'], 'target_from': ..."
4,2,0,0,[]
5,4,0,0,[]
6,14,0,0,[]
7,6,0,0,[]
8,8,27,2,"[{'target_phrase': ['Gersa'], 'target_from': '..."
9,1,0,0,[]


In [ ]:
#lexeme_df["emlap_instances"] = lexeme_df.apply(lambda row: concordance_for_target_across_sentences(conn, row["target_canonical"], row["target_relemmatized"], window=10).to_dict("records"), axis=1)

In [ ]:
#lexeme_df["emlap_instances"] = lexeme_df["instance_fname"].apply(read_hits)

In [ ]:
#lexeme_df["emlap_instances_N"] = lexeme_df["emlap_instances"].apply(len)

In [ ]:
#lexeme_df.sort_values("emlap_instances_N", ascending=False)[:20]

In [ ]:
#lexeme_df["instances_ids"] = lexeme_df["emlap_instances"].apply(lambda x: [ins["grela_id"][6:] for ins in x])

In [ ]:
emlap_id_title_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['working_title']))
emlap_id_date_dict = dict(zip(emlap_metadata['no.'].astype(str), emlap_metadata['date_publication']))

In [ ]:
def add_emlap_metadata(instances_dict_list):
    instances_dict_list_new = []
    for instance_dict in instances_dict_list:
        id = instance_dict["grela_id"][6:]
        instance_dict["title"] = emlap_id_title_dict[id]
        instance_dict["year"] = emlap_id_date_dict[id]
        instances_dict_list_new.append(instance_dict)
    return instances_dict_list_new

In [ ]:
# add metadata to instances column
lexeme_df["emlap_instances"] = lexeme_df["emlap_instances"].apply(add_emlap_metadata)

In [ ]:
# export instances also into CSV...
csv_path = "../data/emlap_ruland_instances_csv/"
os.makedirs(csv_path, exist_ok=True)

def create_csvs(fname):
        path = os.path.join(outdir, fname)
        instances_df = pd.read_parquet(path)
        print(len(instances_df))
        #instances_dict_list = instances_df.to_dict("records")
        #instances_dict_list_new = add_emlap_metadata(instances_dict_list)
        #instances_df_new = pd.DataFrame(instances_dict_li# st_new)
        fpath = os.path.join(csv_path, fname.replace(".parquet", ".csv"))
        #instances_df_new.to_csv(fpath)
        instances_df.to_csv(fpath)

In [ ]:
for fname in lexeme_df["instance_fname"]:
    create_csvs(fname)

In [32]:
lexeme_df.head(5)

,Lemma,lemmas_all,target_canonical,entry_ids,entry_count,types,variants,notes,senses_flat,sense_count,sense_def_all,sense_def_strict_all,sense_raw_all,translations_all,target_len,target_relemmatized,instance_fname,emlap_instances,emlap_instances_N,instances_ids
0,Naphtha,[Naphtha],naphtha,[Ruland1612-Naphtha],1,[N],[],[],"[{'def': 'id est, pir, ignis.; id est, flatus ...",15,"[id est, pir, ignis.; id est, flatus minerae, ...","[id est, pir, ignis.; id est, flatus minerae, ...",[Iudaicum bitumen diximus esse speciem Naphtha...,"[{'entry_id': 'Ruland1612-Naphtha', 'quote': '...",1,naphtha,naphtha.parquet,"[{'target_phrase': ['naphtha'], 'target_from':...",6,"[100012, 100012, 100012, 100038, 100063, 100070]"
1,Baurac,"[Baurac, Bäurac]",baurac,"[Ruland1612-Baurac, Ruland1612-Baurac, Ruland1...",10,[B],[],[],"[{'def': None, 'entry_id': 'Ruland1612-Baurac'...",10,"[wird vom Schaum deß Glases oder Nitere., id e...","[None, id est, sal gemmae., i. e. id est sapph...","[wird vom Schaum deß Glases oder Nitere., id e...","[{'entry_id': 'Ruland1612-Baurac', 'quote': 'w...",1,baurac,baurac.parquet,[],0,[]
2,Nitrum,[Nitrum],nitrum,"[None, Ruland1612-Nitrum, Ruland1612-Nitrum, R...",9,[N],[],[],"[{'def': 'Baurach, sal petrosum, nitrum à Germ...",9,"[Baurach, sal petrosum, nitrum à German, Fossi...","[Baurach, sal petrosum, nitrum à German, Fossi...","[Baurach, sal petrosum, nitrum à German. Salpe...","[{'entry_id': None, 'quote': 'SalpeterBergsalt...",1,nitrum,nitrum.parquet,"[{'target_phrase': ['Nitra'], 'target_from': '...",488,"[100001, 100001, 100002, 100002, 100005, 10000..."
3,Elixir,[Elixir],elixir,"[Ruland1612-Elixir,-Elei,, Ruland1612-Elixir, ...",6,[E],"[Elei, Xir]",[],"[{'def': 'id est, Medicina.', 'entry_id': 'Rul...",5,"[id est, Medicina., est fermentum, cuius minim...","[id est, Medicina., est fermentum, cuius minim...","[id est, Medicina., est fermentum, cuius minim...","[{'entry_id': 'Ruland1612-Elixir', 'quote': 'i...",1,elixir,elixir.parquet,"[{'target_phrase': ['Elixir'], 'target_from': ...",1064,"[100001, 100007, 100007, 100007, 100007, 10000..."
4,Magnesia,[Magnesia],magnesia,"[Ruland1612-Magnesia, Ruland1612-Magnesia, Rul...",5,[M],[],[],"[{'def': 'id est, testudo vel sulphul.', 'entr...",5,"[id est, testudo vel sulphul., Ist das gem sch...","[id est, testudo vel sulphul., Ist das gem sch...","[id est, testudo vel sulphul., Ist das gem sch...","[{'entry_id': 'Ruland1612-Magnesia', 'quote': ...",1,magnesia,magnesia.parquet,"[{'target_phrase': ['Magnesia'], 'target_from'...",222,"[100003, 100003, 100003, 100003, 100007, 10000..."


In [ ]:
# conservative under Google’s 50k hard limit
MAX_CELL = 48000

def _clip(s: str, n: int = MAX_CELL) -> str:
    return s if len(s) <= n else (s[:n-1] + "…")

def _stringify(v) -> str:
    """Controlled stringification so lists/dicts don't explode."""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return ""
    if isinstance(v, (list, dict)):
        # compact JSON, unicode-friendly
        return json.dumps(v, ensure_ascii=False, separators=(',', ':'))
    return str(v)

def prepare_for_sheets(df: pd.DataFrame, *, drop_heavy: bool = False) -> pd.DataFrame:
    """
    Make a sheets-safe copy:
      - optional: drop very heavy/nested cols
      - stringify lists/dicts in a compact way
      - clip any cell > MAX_CELL
    """
    if drop_heavy:
        df = df.drop(columns=[
            # keep this list lean; tweak as you like
            "senses_flat", "Senses", "Translations", "EntryLevelTranslations"
        ], errors="ignore").copy()
    else:
        df = df.copy()

    # stringify then clip per-cell
    df = df.applymap(_stringify)
    df = df.applymap(lambda s: _clip(s, MAX_CELL))
    return df

# --- example usage ---
# keep everything but serialize/clip:
df_for_sheet = prepare_for_sheets(lexeme_df.drop(["emlap_instances"], axis=1), drop_heavy=False)

In [ ]:
set_with_dataframe(ruland_gs.add_worksheet("ruland_emlap_instances_2025-12-12", 1,1), df_for_sheet.sort_values("emlap_instances_N", ascending=False))

In [ ]:
lexeme_df.to_json("../data/large_files/ruland-emlap-grela.json")

In [ ]:
lexeme_df.to_parquet("../data/large_files/ruland-emlap-grela.parquet")

In [ ]:
outdir_fullgrela = ("../data/large_files/grela_ruland_instances/")

In [ ]:
for i, row in lexeme_df.iterrows():
    tc = row.get("target_canonical")
    tr = row.get("target_relemmatized")
    target = tc or tr
    name = safe_name(tc)
    fname = f"{name}.parquet"
    if fname not in done:
        fpath = str(outdir_fullgrela / fname)
            # streams directly to Parquet (no df = ..., no df.to_parquet)
        grela_helpers.concordance_for_target_across_sentences(
            conn,
            tc,
            tr,
            window=10,
            out_path=fpath,
            emlap_only=False,
            heavy_normalization=False,
        )
        print(f"[{i}] saved → {fpath}")
        done.append(fname)